In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torch import from_numpy
from network import NeuralNetwork
from preprocessor import create_dataset
import numpy as np
import torch.utils.data as data

In [ ]:
model = NeuralNetwork().double()
device = torch.device("cpu")

In [ ]:
transform = None
target_transform = None

In [ ]:
train_set = create_dataset(train=True)
test_set = create_dataset(train=False)
batch_size = 64

In [ ]:
train_set_size = int(len(train_set) *0.8)
valid_set_size = len(train_set) - train_set_size
train_set, valid_set = data.random_split(
    train_set, [train_set_size, valid_set_size])


In [ ]:
train_dataloader = DataLoader(
    train_set, batch_size=batch_size, drop_last=True)
valid_dataloader = DataLoader(
    valid_set, batch_size=batch_size, drop_last=True)
test_dataloader = DataLoader(
    test_set, batch_size=batch_size, drop_last=True)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        #computing prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 25 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
            
def valid_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    valid_loss, correct = 0, 0
    

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            valid_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    valid_loss /= num_batches
    correct /= size
    accuracy = 100*correct
    global best_valid_accuracy
#     global best_valid_loss
    global counter
#     if loss<best_valid_loss:
    if accuracy>best_valid_accuracy:
        counter = 0
#         best_valid_loss = valid_loss
        best_valid_accuracy = accuracy
        torch.save(model.state_dict(), 'best-model-parameters.pt')
    else:
        counter+=1
    print(f"Validation Error: \n Accuracy: {(accuracy):>0.1f}%, Avg loss: \
    {valid_loss:>8f}")
    print(f"Best validation accuracy: {(best_valid_accuracy):>0.1f}% \n")

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    accuracy = 100*correct
    print(f"Test Error: \n Accuracy: {(accuracy):>0.1f}%, Avg loss: \
    {test_loss:>8f}")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
best_valid_accuracy = 0.00
# best_valid_loss = 10
counter = 0


epochs = 1000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    valid_loop(valid_dataloader, model, loss_fn)
    if counter == 150:
        break
print("Done!")

In [ ]:
# Running the trained model on the test data with weights of model which
# achieved best validation accuracy.
model = NeuralNetwork().double()
model.load_state_dict(torch.load('best-model-parameters.pt', map_location='cpu'))
test_loop(test_dataloader, model, loss_fn)

In [ ]:
# Goals to add to this project:
# 1) add label support to dataloaders for confusion matrix purposes
# 2) visualize training on tensorboard
# 3) consider using stratified train test split to split all three sets for
#    possible improved results (use n_splits=1)
#    I should at least use this split for the train data to split into train
#    and val.  I might want to keep the same test set.
# 4) Normalize the incoming data through a transform
# 5) Clean up the pipeline so that all transforms happen through the transform method